# Домашнее задание 2.1. Сверточные сети



В этом задании вы должны:
1. Написать слой Conv2d на Numpy и определить в нем forward-backward методы
2. Определить слой MaxPool2d
3. Написать всю необходимую обвязку для обучения: оптимизатор с адаптивным шагом и класс, позволяющий изменять расписание для learning rate'а



> Обратите внимание, что в этом задании больше нет тестов.
> Вы должны сами проверять свой код.  
> Это можно сделать так:
> 1. Написать юнит-тесты с помощью Pytorch. То есть, ваш модудь должен повторять поведение torch'а
> 2. Проверять архитектуру не на всем датасете, а на подвыборке: при наивной имплементации слоев одна эпоха на всем датасете будет занимать около двух часов

## Numpy-имплементация сверточной нейронной сети


Вставьте сюда имплементацию из первого домашнего задания.



> Обратите внимание, что обновление весов теперь производится с помощью специального класса **Optimizer**



In [2]:
import os
import sys
import numpy as np

def load_mnist(flatten=False):
    """taken from https://github.com/Lasagne/Lasagne/blob/master/examples/mnist.py"""
    # We first define a download function, supporting both Python 2 and 3.
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
    else:
        from urllib.request import urlretrieve

    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print("Downloading %s" % filename)
        urlretrieve(source + filename, filename)

    # We then define functions for loading MNIST images and labels.
    # For convenience, they also download the requested files if needed.
    import gzip

    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the inputs in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        # The inputs are vectors now, we reshape them to monochrome 2D images,
        # following the shape convention: (examples, channels, rows, columns)
        data = data.reshape(-1, 1, 28, 28)
        # The inputs come as bytes, we convert them to float32 in range [0,1].
        # (Actually to range [0, 255/256], for compatibility to the version
        # provided at http://deeplearning.net/data/mnist/mnist.pkl.gz.)
        return data / np.float32(256)

    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the labels in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        # The labels are vectors of integers now, that's exactly what we want.
        return data

    # We can now download and read the training and test set images and labels.
    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

    # We reserve the last 10000 training examples for validation.
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]

    if flatten:
        X_train = X_train.reshape([X_train.shape[0], -1])
        X_val = X_val.reshape([X_val.shape[0], -1])
        X_test = X_test.reshape([X_test.shape[0], -1])

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [3]:
class Layer:
    """
    A building block. Each layer is capable of performing two things:

    - Process input to get output:           output = layer.forward(input)

    - Propagate gradients through itself:    grad_input = layer.backward(input, grad_output)

    Some layers also have learnable parameters which they update during layer.backward.
    """
    def __init__(self):
        """Here you can initialize layer parameters (if any) and auxiliary stuff."""
        pass

    def forward(self, input):
        """
        Takes input data of shape [batch, input_units], returns output data [batch, output_units]
        """
        return input

    def backward(self, input, grad_output):
        """
        Performs a backpropagation step through the layer, with respect to the given input.

        To compute loss gradients w.r.t input, you need to apply chain rule (backprop):

        d loss / d x  = (d loss / d layer) * (d layer / d x)

        Luckily, you already receive d loss / d layer as input, so you only need to multiply it by d layer / d x.

        If your layer has parameters (e.g. dense layer), you also need to update them here using d loss / d layer
        """
        input_dim = input.shape[1]

        d_layer_d_input = np.eye(input_dim)

        return np.dot(grad_output, d_layer_d_input) # chain rule

In [4]:
class ReLU(Layer):
    def __init__(self):
        """ReLU layer simply applies elementwise rectified linear unit to all inputs"""
        pass

    def forward(self, input):
        """Apply elementwise ReLU to [batch, input_units] matrix"""
        output = np.array(input)
        output[output < 0] = 0
        return output

    def backward(self, input, grad_output):
        """Compute gradient of loss w.r.t. ReLU input"""
        relu_grad_mask = np.zeros_like(input)
        relu_grad_mask[input > 0] = 1
        return grad_output * relu_grad_mask

In [ ]:
class Dense(Layer):
    def __init__(self, input_units, output_units, learning_rate=0.1):
        """
        A dense layer is a layer which performs a learned affine transformation:
        f(x) = <W*x> + b
        """
        self.learning_rate = learning_rate

        # initialize weights with small random numbers from normal distribution
        # you can change the intializtion method
        self.weights = np.random.randn(input_units, output_units)*0.01
        self.biases = np.zeros(output_units)

    def forward(self,input):
        """
        Perform an affine transformation:
        f(x) = <W*x> + b

        input shape: [batch, input_units]
        output shape: [batch, output_units]
        """
        return np.dot(input, self.weights) + self.biases

    def backward(self, input, grad_output):

        # compute d f / d x = d f / d dense * d dense / d x
        # where d dense/ d x = weights transposed
        # grad_output is a derivative of the next (in forward) layer of prediction
        # this result is needed for the next layer
        grad_input = np.dot(grad_output, np.transpose(self.weights))

        # compute gradient w.r.t. weights and biases
        grad_weights = np.dot(np.transpose(input), grad_output)
        grad_biases = np.sum(grad_output, axis=0)

        assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape
        # Here we perform a stochastic gradient descent step.
        # or step of another gradient method
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases

        return grad_input

In [152]:
class Conv2d(Layer):
    def __init__(self, input_channels, output_channels, kernel_size, learning_rate=0.1):
        self.weights = np.random.randn(input_channels, output_channels, kernel_size, kernel_size)*0.01
        self.biases = np.zeros(output_channels)

        self.learning_rate = learning_rate
        
        self.grad_weights = None
        self.grad_biases = None

    def _conv(input, weights, biases):
        kernel_height = np.shape(weights)[2]
        kernel_width = np.shape(weights)[3]
        output_chanels = np.shape(weights)[1]
        # Output dimention is determined by the number of filters in the layer.
        # Each filter is a tensor with the size (input_channels, kernel_size, kernel_size)
        # (In this example second axis is an index of filter)
        # Such a filter produces one number per convolution 
        #   and this convolution is performed for each part of the input tensor.
        batch_size = np.shape(input)[0]
        output_height = np.shape(input)[2] - kernel_height + 1
        output_width = np.shape(input)[3] - kernel_width + 1

        # This is the result of convollution of each filter
        filters = np.empty((output_chanels, output_height, output_width, batch_size))
        for filter_idx in range(output_chanels):
            for i in range(output_height):
                for j in range(output_width):
                    filters[filter_idx, i, j] = \
                        np.tensordot(input[:, :, i:i+kernel_height, j:j+kernel_width],
                                     weights[:, filter_idx, :, :],
                                     axes=([1, 2, 3], [0, 1, 2]))
        output = np.rollaxis(filters, 3, 0)
        if biases is not None:
        # Here values from biases are broadcasted
            output += np.reshape(biases, (1, output_chanels, 1, 1))
        return output

    def _pad(tensor, val, new_height, new_width):
        pad_h = (new_height - np.shape(tensor)[2]) // 2
        pad_w = (new_width - np.shape(tensor)[3]) // 2
        padding = ((0, 0), (0, 0), (pad_h, pad_h), (pad_w, pad_w))
        res = np.pad(tensor, padding, 'constant', constant_values=(val, val))
        assert np.shape(res)[2] == new_height and np.shape(res)[3] == new_width
        return res
    
    def _flip_transpose(tensor):
        tensor = np.flip(tensor, (2, 3))
        tensor = np.moveaxis(tensor, 2, 3) # this should work
        return tensor

    def forward(self, input):
        """
        Perform an convolution:

        output_height = input_height - kernel_size + 1
        output_width = input_width - kernel_size + 1

        input shape: [batch, input_channels, input_height, input_width]
        output shape: [batch, output_channels, output_height, output_width]
        """
        return Conv2d._conv(input, self.weights, self.biases)
        

    def backward(self, input, grad_output):
        # Grad output has the shape [batch, output_channels, output_height, output_width]
        # I don't know how this is possible, but if has these axises, convolution just works
        input = np.moveaxis(input, 0, 1)
        grad_weights = Conv2d._conv(input, grad_output, None)
        input = np.moveaxis(input, 0, 1)
        # dldb = dldz * dzdb 
        # dzdb = 1
        # dldz = grad_output
        # Each biases' component is applied to each element with the fixed output_chanels axis,
        #   so input gradient is the sum of all output gradients
        grad_biases = np.sum(grad_output, axis=(0, 2, 3))

        kernel_size = np.shape(self.weights)[2]
        # Probably I shouldn't change input tensor, but whatever
        grad_padded = Conv2d._pad(grad_output, 0,
                                   np.shape(input)[2] + kernel_size - 1,
                                   np.shape(input)[3] + kernel_size - 1)
        tmp_weights = np.copy(self.weights)
        tmp_weights = Conv2d._flip_transpose(tmp_weights)
        tmp_weights = np.moveaxis(tmp_weights, 0, 1)
        grad_input = Conv2d._conv(grad_padded, tmp_weights, None)

        assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape

        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases

        return grad_input

In [146]:
# Conv2D test
from torch import nn
import torch
test_batch_size = 10
test_input_chanels = 3
test_output_chanels = 2
test_input_height = 4
test_input_width = 5
test_kernel_size = 2

test_conv = Conv2d(input_channels=test_input_chanels, 
                   output_channels=test_output_chanels, 
                   kernel_size=test_kernel_size)
test_input = np.random.randn(test_batch_size, 
                             test_input_chanels, 
                             test_input_height,
                             test_input_width)
test_output = test_conv.forward(test_input)
test_layer_torch = nn.Conv2d(in_channels=test_input_chanels,
                     out_channels=test_output_chanels,
                     kernel_size=(test_kernel_size, test_kernel_size))
test_ref = test_layer_torch(torch.tensor(test_input, dtype=torch.float))
print("Test shapes:")
print(test_ref.shape)
print(test_output.shape)
#assert torch.equal(torch.tensor(test_output, dtype=torch.float), test_ref)


Test shapes:
torch.Size([10, 2, 3, 4])
(10, 2, 3, 4)


In [147]:
print(f"Input shapes {np.shape(test_input)}, {np.shape(test_output)}")
test_conv.backward(test_input, test_output)

Input shapes (10, 3, 4, 5), (10, 2, 3, 4)
grad_output shape before pad (10, 2, 3, 4)
input (10, 2, 3, 4), 5, 6
grad_output shape after pad (10, 2, 5, 6)
weights shape before flip (3, 2, 2, 2)
weights shape after flip (3, 2, 2, 2)


array([[[[-5.11001667e-04, -6.31551106e-04, -8.68758445e-04,
          -1.01711396e-03,  1.83748802e-04],
         [ 1.11976809e-04, -1.68039207e-03, -1.35932624e-03,
          -6.67949680e-05, -1.28401713e-04],
         [-2.22100701e-03, -7.35242328e-04,  2.83525726e-04,
          -7.78658135e-05, -1.39136373e-04],
         [ 5.84763765e-04,  5.07679891e-04,  4.18121853e-04,
           9.94755519e-05, -4.94088685e-05]],

        [[-4.72519659e-04, -8.48570891e-04, -2.36693272e-04,
           8.96231402e-04,  6.02728103e-04],
         [-1.74804067e-03, -1.77697210e-03, -1.47159473e-03,
          -1.83497416e-03, -3.69348485e-04],
         [-5.47330468e-04,  7.56472350e-04,  4.49689504e-04,
           1.93732843e-04,  4.94583064e-04],
         [-1.73263845e-04,  6.47217413e-04, -1.56549619e-04,
           7.43344269e-05, -1.29729237e-04]],

        [[ 5.96201736e-04,  2.17971130e-04, -1.46795449e-04,
          -1.30643561e-04, -2.49134161e-04],
         [ 9.57775149e-04,  9.18301863e-04

In [117]:
!wget --quiet --show-progress "https://raw.githubusercontent.com/mryab/dl-hse-ami/master/week01_intro/util.py"

util.py.1           100%[===================>]   3,97K  --.-KB/s    in 0s      


In [151]:
# some tests
from util import eval_numerical_gradient

grads = test_conv.backward(test_input, test_output)
numeric_grads = eval_numerical_gradient(lambda x: test_conv.forward(x).mean(), x=test_input)
assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0),\
    "gradient returned by your layer does not match the numerically computed gradient"

grad_output shape before pad (10, 2, 3, 4)
input (10, 2, 3, 4), 5, 6
grad_output shape after pad (10, 2, 5, 6)
weights shape before flip (3, 2, 2, 2)
weights shape after flip (3, 2, 2, 2)
my grads [[[[ 0.02970716  0.02257574]
   [-0.0030515   0.05990383]]

  [[ 0.02672301  0.03915883]
   [ 0.09213424  0.07432515]]]


 [[[ 0.106251   -0.0375867 ]
   [-0.13800736  0.00326983]]

  [[ 0.03884403 -0.03407668]
   [-0.12116685 -0.13928758]]]]
real grads [[[[-0.0012761   0.00652251]
   [ 0.0001263   0.00961945]]

  [[ 0.00737544  0.00829457]
   [ 0.00986708  0.01001397]]]


 [[[-0.0012761   0.00652251]
   [ 0.0001263   0.00961945]]

  [[ 0.00737544  0.00829457]
   [ 0.00986708  0.01001397]]]]


AssertionError: gradient returned by your layer does not match the numerically computed gradient

In [ ]:
class MaxPool2d(Layer):
    def __init__(self, kernel_size):
        self.kernel_size = kernel_size


    def forward(self, input):
        """
        Perform an pooling:

        output_height = input_height // kernel_size
        output_width = input_width // kernel_size

        input shape: [batch, input_channels, input_height, input_width]
        output shape: [batch, input_channels, output_height, output_width]
        """
        #<your code here>

    def backward(self, input, grad_output):
        #grad_input = <your code here>

        return grad_input

In [ ]:
class Flatten(Layer):
    def forward(self, input):
          """
          Perform an flatten operation:

          input shape: [batch, input_channels, input_height, input_width]
          output shape: [batch, input_channels * output_height * output_width]
          """
          #<your code here>

    def backward(self, input, grad_output):
        #grad_input = <your code here>

        return grad_input

In [ ]:
def softmax_crossentropy_with_logits(logits,reference_answers):
    """Compute crossentropy from logits[batch,n_classes] and ids of correct answers"""
    logits_for_answers = logits[np.arange(len(logits)),reference_answers]

    xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))

    return xentropy

def grad_softmax_crossentropy_with_logits(logits,reference_answers):
    """Compute crossentropy gradient from logits[batch,n_classes] and ids of correct answers"""
    ones_for_answers = np.zeros_like(logits)
    ones_for_answers[np.arange(len(logits)),reference_answers] = 1

    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)

    return (- ones_for_answers + softmax) / logits.shape[0]

## Имплементация оптимизатора и изменения learning rate'a

В имплементации этих двух классов есть небольшие неточности.
Посмотрите, как сделана имплементация метода моментов в Pytorch и добавьте пропущенное.

> Добавлять моменты Нестерова не нужно!



In [ ]:
class SGDOptimizer:
    def __init__(self, momentum=0.9, dampening=0.0, weight_decay=0.0):
        """
        Wrapper which perfoms weights update
        """
        self.momentum = momentum
        self.dampening = dampening
        self.weight_decay = weight_decay

        # здесь будем копить моментумы
        self.momentum_buffer = 0

    def step(self, weights, grad_weights, lr=0.1):
      """
      Update weights
      """
      self.momentum_buffer = self.momentum * self.momentum_buffer + (1 - self.dampening) * grad_weights

      #<your code here>

      return weights - lr * self.momentum_buffer

In [ ]:
class LRScheduler:
    def __init__(self, lr):
        """
        Wrapper which perfoms learning rate updates
        """
        self.lr = lr
        self.current_step = 0

    def get_lr(self, step):
      """
      Update learing rate for current iteration
      """
      current_lr = self.lr / (self.current_step + 1)
      #<your code here>
      self.current_step += 1
      return current_lr

## Итоговая нейросеть

Все готово для запуска нейросети. Нейросеть будем тестировать на классическом датасете MNIST. Код ниже визуализирует несколько примеров из этого датасета.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

X_train, y_train, X_val, y_val, X_test, y_test = load_mnist(flatten=False)

plt.figure(figsize=[6,6])
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.title("Label: %i"%y_train[i])
    plt.imshow(X_train[i].reshape([28,28]),cmap='gray');

В нашей реализации сеть - просто список (Python-list) слоев.



> Обратите внимание, что у нас нет глобального пулинга. При изменении архитектуры сети вы должны поменять входую размерность в последнем Dense слое



In [ ]:
network = []
network.append(Conv2d(1, 4, 5))
network.append(MaxPool2d(2))
network.append(ReLU())
network.append(Conv2d(4, 8, 5))
network.append(MaxPool2d(2))
network.append(ReLU())
network.append(Flatten())
network.append(Dense(5 * 5 * 8, 10))

In [ ]:
learning_rate = 0.1

optimizer = SGDOptimizer()
scheduler = LRScheduler(learning_rate)

Реализуйте прямой проход по целой сети, последовательно вызывая .forward() для каждого слоя.

In [ ]:
def forward(network, X):
    """
    Compute activations of all network layers by applying them sequentially.
    Return a list of activations for each layer.
    Make sure last activation corresponds to network logits.
    """
    activations = []
    input = X

    # <your code here>

    assert len(activations) == len(network)
    return activations

def predict(network, X):
    """
    Use network to predict the most likely class for each sample.
    """
    logits = forward(network, X)[-1]
    return logits.argmax(axis=-1)

In [ ]:
def train(network,X,y):
    """
    Train your network on a given batch of X and y.
    You first need to run forward to get all layer activations.
    Then you can run layer.backward going from last to first layer.

    After you called backward for all layers, all Dense layers have already made one gradient step.
    """

    # Get the layer activations
    layer_activations = forward(network,X)
    layer_inputs = [X] + layer_activations  #layer_input[i] is an input for network[i]
    logits = layer_activations[-1]

    # Compute the loss and the initial gradient
    loss = softmax_crossentropy_with_logits(logits,y)
    loss_grad = grad_softmax_crossentropy_with_logits(logits,y)

    # propagate gradients through network layers using .backward
    # hint: start from last layer and move to earlier layers
    #<YOUR CODE>

    # update weights and biases with optimizer
    #<YOUR CODE>

    # update learning rate
    #<YOUR CODE>

    return np.mean(loss)

Все готово для запуска обучения. Если все реализовано корректно, то точность классификации на валидационном множестве **должна быть около** 99%.

In [ ]:
from tqdm.auto import tqdm
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in tqdm(range(0, len(inputs) - batchsize + 1, batchsize)):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
from IPython.display import clear_output
train_log = []
val_log = []

In [ ]:
for epoch in range(15):

    for x_batch,y_batch in iterate_minibatches(X_train, y_train, batchsize=32, shuffle=True):
        train(network, x_batch, y_batch)

    train_log.append(np.mean(predict(network, X_train) == y_train))
    val_log.append(np.mean(predict(network, X_val) == y_val))

    clear_output()
    print("Epoch",epoch)
    print("Train accuracy:",train_log[-1])
    print("Val accuracy:",val_log[-1])
    plt.plot(train_log,label='train accuracy')
    plt.plot(val_log,label='val accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.show()
